In [190]:
import requests
import pandas as pd
import os
import zipfile

pd.options.display.max_rows=200
pd.options.display.max_columns=100

In [211]:
# from https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests
def download_file(url, local_path):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
    print('Downloaded to', local_path)
    if (local_path[-4:] == '.zip'):
        with zipfile.ZipFile(local_path, 'r') as f:
            f.extractall(local_path[:-4])
        print('Unzipped to ' + local_path[:-4])

In [283]:
"""
FEC bulk data files

https://www.fec.gov/data/browse-data/?tab=bulk-data

Data files of interest:
- Candidate master - cn20.zip - Records for candidates registered w/ FEC or on ballot
- Committee master - one record for each committee registerd w/ FEC (federal PACS, party committess, campaign committes, etc.)
    - Use this for "totals" info
- Contributions by individuals
    - Contributions by INDIVIDUALS to FEDERAL COMMITTEES
- Contributions from COMMITTEES to CANDIDATES & INDEPENDENT EXPENDITURES 
- Operating expenditures

"""

def get_mt_federal_candidates():
    url = 'https://www.fec.gov/files/bulk-downloads/2020/cn20.zip'
    local_path = 'data/candidate-list-20.zip'
    download_file(url, local_path)
    names = ["CAND_ID", "CAND_NAME", "CAND_PTY_AFFILIATION", "CAND_ELECTION_YR", "CAND_OFFICE_ST", "CAND_OFFICE", "CAND_OFFICE_DISTRICT", "CAND_ICI", "CAND_STATUS", "CAND_PCC", "CAND_ST1", "CAND_ST2", "CAND_CITY", "CAND_ST", "CAND_ZIP"]
    df = pd.read_csv(local_path, delimiter="|", header=None, names=names)
    mt_2020 = df[
        (df['CAND_OFFICE_ST'] == 'MT') 
        & (df['CAND_ELECTION_YR'] == 2020)
        & (df['CAND_OFFICE'].isin(['H','S'])) # House or Senate
    ]
    mt_2020.to_json('data/mt-2020-candidates.json', orient='records')
    return mt_2020

def get_campaign_summaries(mt_2020_candidate_ids):
    # JUST for 2019-20 period
    url = 'https://www.fec.gov/files/bulk-downloads/2020/webl20.zip'
    local_path = 'data/candidate-committee-summaries-20.zip'
    download_file(url, local_path)
    names = ["CAND_ID", "CAND_NAME", "CAND_ICI", "PTY_CD", "CAND_PTY_AFFILIATION", "TTL_RECEIPTS", "TRANS_FROM_AUTH", "TTL_DISB", "TRANS_TO_AUTH", "COH_BOP", "COH_COP", "CAND_CONTRIB", "CAND_LOANS", "OTHER_LOANS", "CAND_LOAN_REPAY", "OTHER_LOAN_REPAY", "DEBTS_OWED_BY", "TTL_INDIV_CONTRIB", "CAND_OFFICE_ST", "CAND_OFFICE_DISTRICT", "SPEC_ELECTION", "PRIM_ELECTION", "RUN_ELECTION", "GEN_ELECTION", "GEN_ELECTION_PRECENT", "OTHER_POL_CMTE_CONTRIB", "POL_PTY_CONTRIB", "CVG_END_DT", "INDIV_REFUNDS", "CMTE_REFUNDS"]
    df = pd.read_csv(local_path, delimiter="|", header=None, names=names)
    df = df[df['CAND_ID'].isin(mt_2020_candidate_ids)]
    df.to_json('data/mt-2020-candidate-committee-summaries.json', orient='records')
    return df
    

# mt_2020_candidates = get_mt_federal_candidates()
mt_2020_candidate_committees = list(mt_2020_candidates['CAND_PCC'])
mt_2020_candidate_ids = list(mt_2020_candidates['CAND_ID'])
mt_2020_candidate_summaries = get_campaign_summaries(mt_2020_candidate_ids)

Downloaded to data/candidate-committee-summaries-20.zip
Unzipped to data/candidate-committee-summaries-20


In [285]:
mt_2020_candidate_summaries

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
618,H0MT00090,"WINTER, THOMAS",C,1,DEM,197671.00,0.00,84020.79,0.00,0.00,113650.21,6259.50,0.00,0.0,0.0,0.0,0.00,185911.50,MT,0,NaN,NaN,NaN,NaN,NaN,5000.00,0.0,09/30/2019,0.00,0.00
619,H0MT00116,"RAINS, MATTHEW",C,1,DEM,35972.00,0.00,21424.45,0.00,0.00,14547.55,0.00,0.00,5000.0,0.0,0.0,41407.82,30972.00,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.00
620,H8MT01232,"WILLIAMS, KATHLEEN",C,1,DEM,829824.27,35820.26,239171.21,35820.26,36957.25,627610.31,0.00,0.00,0.0,0.0,0.0,0.00,754799.01,MT,0,NaN,NaN,NaN,NaN,NaN,11500.00,0.0,09/30/2019,3661.00,0.00
621,H0MT00108,"DOOLING, JOE",C,2,REP,14297.57,0.00,5136.13,0.00,0.00,9161.44,0.00,0.00,0.0,0.0,0.0,0.00,14297.57,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.00
622,H0MT01098,"LAMM, DEBRA",C,2,REP,35312.27,0.00,3063.77,0.00,0.00,32248.50,2810.02,0.00,0.0,0.0,0.0,0.00,32502.25,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.00
623,H4MT00050,"ROSENDALE, MATT MR.",C,2,REP,721016.87,97.18,139823.99,0.00,76178.38,657371.26,0.00,0.00,0.0,0.0,0.0,157831.00,618653.39,MT,0,NaN,NaN,NaN,NaN,NaN,32089.23,0.0,09/30/2019,2700.00,0.00
624,H4MT01033,"STAPLETON, COREY",C,2,REP,122269.76,0.00,38348.78,0.00,0.00,83920.98,0.00,0.00,0.0,0.0,0.0,0.00,122269.76,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.00
625,H8MT01182,"GIANFORTE, GREG",I,2,REP,359696.49,72825.36,338388.32,0.00,31079.69,52387.86,0.00,0.00,0.0,0.0,0.0,500000.00,67430.36,MT,0,NaN,NaN,NaN,NaN,NaN,131475.00,50.0,09/30/2019,2162.10,25597.79
1474,S0MT00090,"COLLINS, WILMOT JAMES",C,1,DEM,177488.76,0.00,128874.45,0.00,0.00,48614.31,0.00,0.00,0.0,0.0,0.0,0.00,175449.90,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,4300.00,0.00
1475,S0MT00108,"MUES, JOHN",C,1,DEM,101394.93,0.00,50200.97,0.00,0.00,51193.96,0.00,18405.67,0.0,0.0,0.0,54056.39,82989.26,MT,0,NaN,NaN,NaN,NaN,NaN,0.00,0.0,09/30/2019,0.00,0.00


In [280]:
mt_2020_candidates

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
640,H0MT00090,"WINTER, THOMAS",DEM,2020,MT,H,0.0,C,C,C00700963,P.O. BOX 8248,NaN,MISSOULA,MT,59807.0
641,H0MT00108,"DOOLING, JOE",REP,2020,MT,H,0.0,C,N,C00709923,3855 CEDAR VALLEY RD,NaN,HELENA,MT,59602.0
642,H0MT00116,"RAINS, MATTHEW",DEM,2020,MT,H,0.0,C,C,C00710509,PO BOX 1710,NaN,GREAT FALLS,MT,59403.0
643,H0MT01080,"JOHNSON, TIMOTHY ALAN",REP,2020,MT,H,0.0,C,N,C00711002,PO BOX 1088,NaN,CORVALLIS,MT,59828.0
644,H0MT01098,"LAMM, DEBRA",REP,2020,MT,H,0.0,C,N,C00718585,PO BOX 1915,NaN,LIVINGSTON,MT,59047.0
1620,H4MT00050,"ROSENDALE, MATT MR.",REP,2020,MT,H,0.0,C,N,C00548289,PO BOX 4907,NaN,HELENA,MT,596044907.0
1621,H4MT01033,"STAPLETON, COREY",REP,2020,MT,H,0.0,C,C,C00709915,2015 EASTRIDGE DR,NaN,BILLINGS,MT,591027904.0
2821,H8MT01182,"GIANFORTE, GREG",REP,2020,MT,H,0.0,I,C,C00631945,1320 MANLEY RD,NaN,BOZEMAN,MT,597158779.0
2822,H8MT01232,"WILLIAMS, KATHLEEN",DEM,2020,MT,H,0.0,C,C,C00701748,PO BOX 548,NaN,BOZEMAN,MT,59771.0
4873,S0MT00090,"COLLINS, WILMOT JAMES",DEM,2020,MT,S,0.0,C,C,C00706317,PO BOX 458,NaN,HELENA,MT,59624.0


In [40]:
def get_campaign_summaries():
    # Summary for current campaigns
    # documentation: https://www.fec.gov/campaign-finance-data/all-candidates-file-description/
    url = 'https://www.fec.gov/files/bulk-downloads/2020/webl20.zip'
    local_path = 'data/current20.zip'
    names = ['CAND_ID', 'CAND_NAME', 'CAND_ICI', 'PTY_CD', 'CAND_PTY_AFFILIATION', 'TTL_RECEIPTS', 'TRANS_FROM_AUTH',
            'TTL_DISB', 'TRANS_TO_AUTH', 'COH_BOP', 'COH_COP', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS', 
             'CAND_LOAN_REPAY', 'OTHER_LOAN_REPAY', 'DEBTS_OWED_BY', 'TTL_INDIV_CONTRIB', 
             'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT', 'SPEC_ELECTION', 'PRIM_ELECTION', 'RUN_ELECTION', 'GEN_ELECTION',
             'GEN_ELECTION_PRECENT', 'OTHER_POL_CMTE_CONTRIB', 'POL_PTY_CONTRIB', 'CVG_END_DT', 'INDIV_REFUNDS', 'CMTE_REFUNDS'
            ]
    download_file(url, local_path)
    current = pd.read_csv(local_path, delimiter="|", header=None, names=names)
    current = current[current['CAND_OFFICE_ST'] == 'MT']
    current.drop(labels=['SPEC_ELECTION','PRIM_ELECTION','RUN_ELECTION','GEN_ELECTION','GEN_ELECTION_PRECENT'], axis=1, inplace=True)
    return current

current = get_campaign_summaries()

Written to data/current20.zip


In [42]:
current

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,COH_COP,CAND_CONTRIB,CAND_LOANS,OTHER_LOANS,CAND_LOAN_REPAY,OTHER_LOAN_REPAY,DEBTS_OWED_BY,TTL_INDIV_CONTRIB,CAND_OFFICE_ST,CAND_OFFICE_DISTRICT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
459,H0MT00090,"WINTER, THOMAS",C,1,DEM,133635.00,0.00,29789.93,0.00,0.00,103845.07,234.00,0.0,0.0,0.0,0.0,0.0,128401.00,MT,0,5000.00,0.0,06/30/2019,0.00,0.00
460,H8MT01232,"WILLIAMS, KATHLEEN",C,1,DEM,446342.79,35820.26,133214.35,35820.26,36957.25,350085.69,0.00,0.0,0.0,0.0,0.0,0.0,394967.53,MT,0,0.00,0.0,07/31/2019,186.00,0.00
461,H4MT00050,"ROSENDALE, MATT MR.",C,2,REP,269179.17,97.18,40441.05,0.00,76178.38,304916.50,0.00,0.0,0.0,0.0,0.0,157831.0,192418.56,MT,0,6500.00,0.0,06/30/2019,2700.00,0.00
462,H4MT01033,"STAPLETON, COREY",C,2,REP,108014.00,0.00,25.00,0.00,0.00,107989.00,0.00,0.0,0.0,0.0,0.0,0.0,108014.00,MT,0,0.00,0.0,06/30/2019,0.00,0.00
463,H8MT01182,"GIANFORTE, GREG",I,2,REP,357457.35,72825.36,322806.70,0.00,31079.69,65730.34,0.00,0.0,0.0,0.0,0.0,500000.0,67430.36,MT,0,131475.00,50.0,06/30/2019,1162.10,25597.79
1098,S0MT00090,"COLLINS, WILMOT JAMES",C,1,DEM,91764.37,0.00,39138.31,0.00,0.00,52626.06,1064.86,0.0,0.0,0.0,0.0,0.0,90699.51,MT,0,0.00,0.0,06/30/2019,0.00,0.00
1099,S2MT00096,"DAINES, STEVEN",I,2,REP,2641188.43,422388.20,581045.41,0.00,1450176.34,3510319.36,0.00,0.0,0.0,0.0,0.0,0.0,1417549.82,MT,0,794692.49,0.0,06/30/2019,15472.75,1200.00


In [287]:
def get_individual_contributions():
    # bulk download individual contributions
    url = 'https://www.fec.gov/files/bulk-downloads/2020/indiv20.zip'
    local_path = 'data/individual-contributions-20.zip'
    if not os.path.exists(local_path): download_file(url, local_path) # for testing
    names = ["CMTE_ID", "AMNDT_IND", "RPT_TP", "TRANSACTION_PGI", "IMAGE_NUM", "TRANSACTION_TP", "ENTITY_TP", "NAME", "CITY", "STATE", "ZIP_CODE", "EMPLOYER", "OCCUPATION", "TRANSACTION_DT", "TRANSACTION_AMT", "OTHER_ID", "TRAN_ID", "FILE_NUM", "MEMO_CD", "MEMO_TEXT", "SUB_ID"]
    dtype = {'ZIP_CODE': str, 'TRANSACTION_DT': str}
    individual = pd.read_csv('./data/individual-contributions-20/itcont.txt', dtype=dtype, delimiter="|", header=None, names=names)
    mt_committees = list(mt_2020['CAND_PCC'])
    mt_indv = individual[individual['CMTE_ID'].isin(mt_committees)].copy()
    mt_indv = mt_indv.merge(mt_2020[['CAND_PCC','CAND_NAME']], left_on='CMTE_ID', right_on='CAND_PCC')
    return mt_indv

# THIS IS NOT WORKING AS EXPECTED
def get_committee_contributions():
    # bulk download committee contributions
    url = 'https://www.fec.gov/files/bulk-downloads/2020/oth20.zip'
    local_path = 'data/committee-contributions-20.zip'
    if not os.path.exists(local_path): download_file(url, local_path) # for testing
    names = ["CMTE_ID", "AMNDT_IND", "RPT_TP", "TRANSACTION_PGI", "IMAGE_NUM", "TRANSACTION_TP", "ENTITY_TP", "NAME", "CITY", "STATE", "ZIP_CODE", "EMPLOYER", "OCCUPATION", "TRANSACTION_DT", "TRANSACTION_AMT", "OTHER_ID", "CAND_ID", "TRAN_ID", "FILE_NUM", "MEMO_CD", "MEMO_TEXT", "SUB_ID"]
    dtype = {'ZIP_CODE': str, 'TRANSACTION_DT': str}
    donations = pd.read_csv('data/committee-contributions-20/itoth.txt', dtype=dtype, delimiter="|", header=None, names=names)
#     return donations
    # filter to dontations received by MT Candidate committees
    mt_committees = list(mt_2020['CAND_PCC'])
    mt_comm = donations[donations['CMTE_ID'].isin(mt_committees)].copy()
    mt_comm = mt_comm.merge(mt_2020[['CAND_PCC','CAND_NAME']], left_on='CMTE_ID', right_on='CAND_PCC')
    return mt_comm

In [288]:
itemized = get_individual_contributions()

In [292]:
itemized.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_PCC,CAND_NAME
0,C00491357,N,Q1,P,201904159146338053,15,IND,"INDRELAND, ILEANA",BOZEMAN,MT,597154795,DELANEY & CO.,REAL ESTATE BROKER,03312019,2800,NaN,ADFC49698138941DB88D,1326203,NaN,NaN,4041820191646716331,C00491357,"DAINES, STEVEN"
1,C00491357,N,Q1,P,201904159146338052,15,IND,"WALLING, GLADYS",WINIFRED,MT,594890055,NONE,RETIRED,03252019,100,NaN,A015D0AF2A3AF4AE39C0,1326203,NaN,NaN,4041820191646716325,C00491357,"DAINES, STEVEN"
2,C00491357,N,Q1,P,201904159146338052,15,IND,"MURPHY, RICHARD B",FAIRFAX,VA,220370001,R.B. MURPHY & ASSOCIATES,PRESIDENT,03112019,1000,NaN,A3A70E45FF2634DEE8D6,1326203,NaN,NaN,4041820191646716327,C00491357,"DAINES, STEVEN"
3,C00491357,N,Q1,G,201904159146338052,15,IND,"THOMA, CARL D",CHICAGO,IL,606112190,THOMA BRAVO,EXECUTIVE,03232019,2800,NaN,A589356CD6DA743B498E,1326203,NaN,NaN,4041820191646716329,C00491357,"DAINES, STEVEN"
4,C00491357,N,Q1,P,201904159146337910,15,IND,"BILLION, JOE",BOZEMAN,MT,597188706,BILLION AUTO SALES,OWNER,02062019,2700,NaN,ADE03926611B7443C882,1326203,NaN,NaN,4041820191646715477,C00491357,"DAINES, STEVEN"


In [290]:
itemized.groupby(['CMTE_ID', 'CAND_NAME']).agg({'CMTE_ID': 'count', 'TRANSACTION_AMT': sum, 'RPT_TP': 'unique'})

,,CMTE_ID,TRANSACTION_AMT,RPT_TP
CMTE_ID,CAND_NAME,,,
C00491357,"DAINES, STEVEN",1079,1224141,"[Q1, Q2]"
C00548289,"ROSENDALE, MATT MR.",117,192234,"[Q1, Q2]"
C00631945,"GIANFORTE, GREG",75,50620,"[Q1, Q2]"
C00700963,"WINTER, THOMAS",133,105669,[Q2]
C00701748,"WILLIAMS, KATHLEEN",430,259505,[Q2]
C00706317,"COLLINS, WILMOT JAMES",98,50685,[Q2]
C00709915,"STAPLETON, COREY",58,105910,[Q2]
C00710509,"RAINS, MATTHEW",36,21925,[Q3]
C00714295,"BALLARD, JACK",44,21440,[Q3]


In [241]:
# individual = get_individual_ccontributions()
committee = get_committee_contributions()

In [245]:
committee.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_PCC,CAND_NAME
0,C00491357,N,Q1,G,201904159146338218,22Z,PAC,CLOUD PEAK ENERGY EMPLOYEE PAC,BROOMFIELD,CO,800218068,NaN,NaN,03312019,200,C00485003,BCF5A71D6683B48FFB32,1326203,NaN,NaN,4041820191646710149,NaN,C00491357,"DAINES, STEVEN"
1,C00491357,N,Q1,P,201904159146338130,18G,COM,SENATE FIREWALL 2020,ALEXANDRIA,VA,223141535,NaN,NaN,03312019,51155,C00696567,AA4B0849ECD594DA9BCB,1326203,NaN,NaN,4041820191646716789,NaN,C00491357,"DAINES, STEVEN"
2,C00491357,N,Q1,P,201904159146338130,18J,PAC,KBR INC PAC,HOUSTON,TX,770027900,NaN,NaN,03302019,1000,C00431114,A597B2C3FC2B14A96B62,1326203,X,NaN,4041820191646716791,NaN,C00491357,"DAINES, STEVEN"
3,C00491357,N,Q1,P,201904159146338130,18J,COM,ENTERPRISE PRODUCTS PARTNERS LP PAC,HOUSTON,TX,770025227,NaN,NaN,03302019,952,C00496752,A9D869F241E3346F5A99,1326203,X,NaN,4041820191646716793,NaN,C00491357,"DAINES, STEVEN"
4,C00491357,N,Q1,P,201904159146338131,15J,IND,"VALACH, KENNETH J",HOUSTON,TX,770243914,TRAMMELL CROW RESIDENTIAL,CEO,03142019,933,NaN,A327AC03E044F4B1A922,1326203,X,NaN,4041820191646716795,NaN,C00491357,"DAINES, STEVEN"


In [232]:
individual.groupby(['CMTE_ID', 'CAND_NAME']).agg({'CMTE_ID': 'count', 'TRANSACTION_AMT': sum, 'RPT_TP': 'unique'})
# THIS IS MISSING Q3 for most candidates?

,,CMTE_ID,TRANSACTION_AMT,RPT_TP
CMTE_ID,CAND_NAME,,,
C00491357,"DAINES, STEVEN",1079,1224141,"[Q1, Q2]"
C00548289,"ROSENDALE, MATT MR.",117,192234,"[Q1, Q2]"
C00631945,"GIANFORTE, GREG",75,50620,"[Q1, Q2]"
C00700963,"WINTER, THOMAS",133,105669,[Q2]
C00701748,"WILLIAMS, KATHLEEN",430,259505,[Q2]
C00706317,"COLLINS, WILMOT JAMES",98,50685,[Q2]
C00709915,"STAPLETON, COREY",58,105910,[Q2]
C00710509,"RAINS, MATTHEW",36,21925,[Q3]
C00714295,"BALLARD, JACK",44,21440,[Q3]


In [246]:
committee.groupby(['CMTE_ID', 'CAND_NAME']).agg({'CMTE_ID': 'count', 'TRANSACTION_AMT': sum, 'RPT_TP': 'unique'})


,,CMTE_ID,TRANSACTION_AMT,RPT_TP
CMTE_ID,CAND_NAME,,,
C00491357,"DAINES, STEVEN",233,853358,"[Q1, Q2]"
C00548289,"ROSENDALE, MATT MR.",2,5097,"[Q1, Q2]"
C00631945,"GIANFORTE, GREG",63,199111,"[Q1, Q2]"
C00701748,"WILLIAMS, KATHLEEN",4,36070,[Q2]
C00710509,"RAINS, MATTHEW",1,5000,[Q3]
C00714295,"BALLARD, JACK",1,3000,[Q3]


In [150]:
donations[donations['CMTE_ID'].isin(mt_committees)]

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
17938,C00631945,N,Q1,G,201904139146114612,24K,CCM,STEVE DAINES FOR MONTANA,HELENA,MT,596241598,NaN,NaN,03312019,2000,C00491357,NaN,BB3E1539F8AFD44D6A29,1324575,NaN,NaN,4041320191646278988
17939,C00631945,N,Q1,P,201904139146114612,24K,CCM,STEVE DAINES FOR MONTANA,HELENA,MT,596241598,NaN,NaN,03312019,2000,C00491357,NaN,B1CB97049813B45E6B49,1324575,NaN,NaN,4041320191646278990
22260,C00631945,N,Q2,P,201907159150857153,24K,COM,BISHOP FOR CONGRESS,CHARLOTTE,NC,282113272,NaN,NaN,06202019,1000,C00699660,H0NC09187,B615F16B1829D40A89A1,1340322,NaN,NaN,4072520191661173673


In [220]:
mt_2020[['CAND_PCC','CAND_NAME']]

,CAND_PCC,CAND_NAME
635,C00700963,"WINTER, THOMAS"
636,C00709923,"DOOLING, JOE"
637,C00710509,"RAINS, MATTHEW"
638,C00711002,"JOHNSON, TIMOTHY ALAN"
639,C00718585,"LAMM, DEBRA"
1613,C00548289,"ROSENDALE, MATT MR."
1614,C00709915,"STAPLETON, COREY"
2814,C00631945,"GIANFORTE, GREG"
2815,C00701748,"WILLIAMS, KATHLEEN"
4866,C00706317,"COLLINS, WILMOT JAMES"


In [158]:
donations[donations['CMTE_ID'] == 'C00711002']

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID


In [168]:
names = ["CMTE_ID", "AMNDT_IND", "RPT_TP", "TRANSACTION_PGI", "IMAGE_NUM", "TRANSACTION_TP", "ENTITY_TP", "NAME", "CITY", "STATE", "ZIP_CODE", "EMPLOYER", "OCCUPATION", "TRANSACTION_DT", "TRANSACTION_AMT", "OTHER_ID", "TRAN_ID", "FILE_NUM", "MEMO_CD", "MEMO_TEXT", "SUB_ID"]
dtype = {'ZIP_CODE': str, 'TRANSACTION_DT': str}
individual = pd.read_csv('./data/individual-contributions-20/itcont.txt', dtype=dtype, delimiter="|", header=None, names=names)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,11,12,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [208]:
# individual[individual['CMTE_ID'] == 'C00700963']

In [70]:
mt_committees = list(mt_2020['CAND_PCC'])
mt_indv = individual[individual['CMTE_ID'].isin(mt_committees)].copy()
mt_indv = mt_indv.merge(mt_2020[['CAND_PCC','CAND_NAME']], left_on='CMTE_ID', right_on='CAND_PCC')

# cleaning
mt_indv['ZIP_CODE'] = mt_indv['ZIP_CODE'].str.slice(0,5)

use_cols = [
    'CAND_NAME','CAND_PCC','CMTE_ID','RPT_TP','TRANSACTION_PGI', 'ENTITY_TP',
    'NAME','CITY','STATE','ZIP_CODE','EMPLOYER','OCCUPATION','TRANSACTION_DT','TRANSACTION_AMT',
    'OTHER_ID','FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID'
]
mt_indv = mt_indv[use_cols]

mt_indv.head()

,CAND_NAME,CAND_PCC,CMTE_ID,RPT_TP,TRANSACTION_PGI,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,"DAINES, STEVEN",C00491357,C00491357,Q1,P,IND,"INDRELAND, ILEANA",BOZEMAN,MT,59715,DELANEY & CO.,REAL ESTATE BROKER,3312019,2800,NaN,1326203,NaN,NaN,4041820191646716331
1,"DAINES, STEVEN",C00491357,C00491357,Q1,P,IND,"WALLING, GLADYS",WINIFRED,MT,59489,NONE,RETIRED,3252019,100,NaN,1326203,NaN,NaN,4041820191646716325
2,"DAINES, STEVEN",C00491357,C00491357,Q1,P,IND,"MURPHY, RICHARD B",FAIRFAX,VA,22037,R.B. MURPHY & ASSOCIATES,PRESIDENT,3112019,1000,NaN,1326203,NaN,NaN,4041820191646716327
3,"DAINES, STEVEN",C00491357,C00491357,Q1,G,IND,"THOMA, CARL D",CHICAGO,IL,60611,THOMA BRAVO,EXECUTIVE,3232019,2800,NaN,1326203,NaN,NaN,4041820191646716329
4,"DAINES, STEVEN",C00491357,C00491357,Q1,P,IND,"BILLION, JOE",BOZEMAN,MT,59718,BILLION AUTO SALES,OWNER,2062019,2700,NaN,1326203,NaN,NaN,4041820191646715477


In [ ]:

#TODO: Filter to these columns

#TODO: Remove duplicates for ACTBLUE etc.
#TODO: Check for reallocations, make sure I understand how committee stuff is being handled
#TODO: Look @ Center for Responsive Politics data products

In [72]:
mt_indv['CAND_NAME'].value_counts()

DAINES, STEVEN           1079
WILLIAMS, KATHLEEN        430
WINTER, THOMAS            133
ROSENDALE, MATT MR.       117
COLLINS, WILMOT JAMES      98
GIANFORTE, GREG            75
STAPLETON, COREY           58
BALLARD, JACK              44
RAINS, MATTHEW             36
Name: CAND_NAME, dtype: int64

In [79]:
# mt_indv[mt_indv['CAND_NAME'] == 'COLLINS, WILMOT JAMES']

In [ ]:
# # Current campaigns
# # documentation: https://www.fec.gov/campaign-finance-data/all-candidates-file-description/


# url = 'https://www.fec.gov/files/bulk-downloads/2020/weball20.zip'
# local_path = 'data/summary20.zip'
# names = ['CAND_ID', 'CAND_NAME', 'CAND_ICI', 'PTY_CD', 'CAND_PTY_AFFILIATION', 'TTL_RECEIPTS', 'TRANS_FROM_AUTH',
#         'TTL_DISB', 'TRANS_TO_AUTH', 'COH_BOP', 'COH_COP', 'CAND_CONTRIB', 'CAND_LOANS', 'OTHER_LOANS', 
#          'CAND_LOAN_REPAY', 'OTHER_LOAN_REPAY', 'DEBTS_OWED_BY', 'TTL_INDIV_CONTRIB', 
#          'CAND_OFFICE_ST', 'CAND_OFFICE_DISTRICT', 'SPEC_ELECTION', 'PRIM_ELECTION', 'RUN_ELECTION', 'GEN_ELECTION',
#          'GEN_ELECTION_PRECENT', 'OTHER_POL_CMTE_CONTRIB', 'POL_PTY_CONTRIB', 'CVG_END_DT', 'INDIV_REFUNDS', 'CMTE_REFUNDS'
#         ]
# download_file(url, local_path)
# df = pd.read_csv(local_path, delimiter="|", header=None, names=names)

In [49]:
# df = pd.read_csv(local_path, delimiter="|", header=None, names=names)
# mt = df[
#     (df['CAND_OFFICE_ST'] == 'MT') 
# #         & (df['CAND_ELECTION_YR'] == 2020)
# #         & (df['CAND_OFFICE'].isin(['H','S'])) # House or Senate
# ]